## Assignment

1. Recommend courses based on programming_language.
   - Build a logic and create flolwchart of it.
   - Write the code for it in python/sql.

## Psudo Solution 

- create_engine
- import data from database 
- preprocessing
- group user table data based on language 
- for user in language
- now create a table with recommendation which have course and user id which are not enrolled by user but they are in that   language  [userid,course]
- now calculate every course avg rating
- for recommendation use avg rating of particular language's course avg rating

In [1]:
import pandas as pd
import sqlalchemy
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine


In [2]:
# cur.execute('select * from rating')
# cur.fetchone()

## creating engine

In [3]:
db_engine = create_engine(
    'mysql+mysqlconnector://{0}:{1}@{2}/{3}'.format(
        'root','9109','localhost','elearning'
    )
)

def extract_table_to_pd(tablename,engine):
    query = f'select * from {tablename}'
    return pd.read_sql(query,engine)
    

## import data from database

In [4]:
ratings = extract_table_to_pd('rating',db_engine)
courses = extract_table_to_pd('courses',db_engine)

## Preprocessing

In [5]:
courses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   course_id             100 non-null    int64 
 1   title                 100 non-null    object
 2   description           100 non-null    object
 3   programming_language  97 non-null     object
dtypes: int64(1), object(3)
memory usage: 3.3+ KB


In [6]:
courses["programming_language"].value_counts()

r               50
python          36
sql              6
shell            3
spreadsheets     2
Name: programming_language, dtype: int64

In [7]:
courses["programming_language"].fillna('r',inplace = True)

In [8]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59172 entries, 0 to 59171
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   user_id    59172 non-null  int64
 1   course_id  59172 non-null  int64
 2   rating     59172 non-null  int64
dtypes: int64(3)
memory usage: 1.4 MB


## group user tabledata based on language

In [9]:
sample_rating = ratings

In [10]:
merged = sample_rating.merge(courses,on = 'course_id')

In [11]:
merged.shape

(59172, 6)

In [12]:
merged_grp_data = merged.groupby(by = ['programming_language'])

In [13]:
# for grp in merged_grp_data.groups.keys():
#     print(merged_grp_data.get_group(grp).head())

In [14]:
merged_grp_data
# merged[merged['user_id'] == 31945]

In [15]:
language_unique = merged_grp_data.groups.keys()
language_unique

dict_keys(['python', 'r', 'shell', 'spreadsheets', 'sql'])

In [16]:
# group11 = merged_grp_data.get_group('python')
# # for user in group11:
# #     print(user)
# group11
# merged.columns

# for user in language

In [16]:
# for user in merged_grp_data.get_group('r')['user_id']:
#     print(user)
#     break

# Iterate through groups
# for group_label, group_data in grouped:
#     print(f"Group: {group_label}")
#     print(group_data)

# merged_grp_data.get_group('r')[:5]
# # course = set(merged_grp_data.get_group('r')['course_id'])
# set(merged_grp_data.get_group('r')[merged_grp_data.get_group('r')['user_id'] == user]['course_id'].values)- {3} 

In [17]:
def unEnrolled():
    new_table = pd.DataFrame()
    for language in language_unique:
    #    group is dataframe having data of one perticular language
        group = merged_grp_data.get_group(language)
    #   course is a set having ids of all courses in a perticular language   
        all_courses = set(group['course_id'])
    #     new table used to append user id and course not enrolled
    
        for user in group['user_id']:
    #         creating set of all courses that already enrolled by user
            enrolled_courses = set(group[group['user_id'] == user]['course_id'].values)
    #     getting not enrolled by set substraction
            not_enrolled = all_courses - enrolled_courses
            not_enrolled = [i for i in not_enrolled]
            not_enrolled_df = pd.DataFrame({
                'user_id': user*len(not_enrolled),
                'course_id' : not_enrolled
            })
    #         print(not_enrolled_df)
            new_table = pd.concat([new_table,not_enrolled_df],ignore_index = True)
    return new_table

In [18]:
# new_table1 = unEnrolled()
# new_table1.to_csv('new_table.csv',index = False)

In [27]:
new_table = pd.read_csv('new_table.csv')
new_table.head()

,user_id,course_id
0,35,1.0
1,35,7.0
2,35,14.0
3,35,17.0
4,35,18.0


In [28]:
# new_table[new_table['user_id'] == 1167212]
avg_ratings = ratings.groupby('course_id',as_index = False).mean()
avg_ratings

,course_id,user_id,rating
0,1,17377.250896,4.537634
1,2,20738.108434,4.253012
2,3,18699.290100,4.572611
3,4,18151.950581,4.305233
4,5,17469.437500,4.281250
...,...,...,...
94,96,18237.636439,4.692765
95,97,17981.387097,4.365591
96,98,17430.888889,4.269006
97,99,16202.086957,4.391304


In [29]:
course_withAvgRating = pd.merge(courses,avg_ratings,on = 'course_id',)

In [30]:
course_withAvgRating.head()

,course_id,title,description,programming_language,user_id,rating
0,1,Machine Learning with Apache Spark,"Spark is a powerful, general purpose tool for ...",python,17377.250896,4.537634
1,2,Financial Analytics in Spreadsheets,Monitoring the evolution of traded assets is k...,spreadsheets,20738.108434,4.253012
2,3,Intermediate R,The intermediate R course is the logical next ...,r,18699.290100,4.572611
3,4,Data Visualization with ggplot2 (Part 2),This ggplot2 tutorial builds on your knowledge...,r,18151.950581,4.305233
4,5,Fraud Detection in R,The Association of Certified Fraud Examiners e...,r,17469.437500,4.281250


In [31]:
new_table['rating'] = [course_withAvgRating[course_withAvgRating['course_id'] == i ]['rating'].values[0] for i in new_table['course_id']]
new_table.to_csv("new_table.csv",index = False)

In [32]:
new_table = pd.read_csv("new_table.csv")
new_table.head()

,user_id,course_id,rating
0,35,1.0,4.537634
1,35,7.0,4.601472
2,35,14.0,4.593220
3,35,17.0,4.147059
4,35,18.0,4.353497


In [33]:
grp_of_user_rec = new_table.groupby('user_id')

In [34]:
final_recommendent = pd.DataFrame(columns=new_table.columns)
for user in grp_of_user_rec.groups.keys():
    temp = grp_of_user_rec.get_group(user)
    final_recommendent = pd.concat([final_recommendent,
                                    temp.sort_values(by='rating',
                                    ascending=False)[:5]],
                                   ignore_index = True)


In [35]:
# grp_of_user_rec.groups.keys()
# grp_of_user_rec.get_group(270)
# pd.DataFrame(new_table.columns)
final_recommendent.to_csv('To_recommendent.csv',index = False)

In [36]:
final_recommendent = pd.read_csv('To_recommendent.csv')
final_recommendent

,user_id,course_id,rating
0,5,96.0,4.692765
1,5,68.0,4.526627
2,5,70.0,4.428144
3,5,15.0,4.313043
4,5,54.0,4.238095
...,...,...,...
203256,1946874,23.0,4.800000
203257,1946874,46.0,4.800000
203258,1946874,24.0,4.653061
203259,1946874,61.0,4.629213
